In [64]:
import pandas as pd
from tqdm import tqdm

from sentence_transformers import SentenceTransformer

from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import numpy as np
from pingouin import mediation_analysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [29]:
def batchify_list(data_list, batch_size):
    """Yield successive n-sized chunks from data_list."""
    for i in range(0, len(data_list), batch_size):
        yield data_list[i:i + batch_size]

def flatten_list(xss):
    return [x for xs in xss for x in xs]

In [110]:
P_d10 = pd.read_csv("./exps/outputs-ZS-validation-n_drops=8-n_sample=-1.csv")
P_d10["n_drops"] = 10
P_d10["label"] = (P_d10["correct_choice_alphabet"] == P_d10["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

accuracy_score(P_d10["parsed_response"], P_d10["correct_choice_alphabet"])

0.0

In [14]:
# Load probing data
P_d10 = pd.read_csv("./exps/outputs-ZS-train-n_drops=10-n_sample=1500.csv")
P_d10["n_drops"] = 10
P_d10["label"] = (P_d10["correct_choice_alphabet"] == P_d10["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P_d8 = pd.read_csv("./exps/outputs-ZS-train-n_drops=8-n_sample=1500.csv")
P_d8["n_drops"] = 8
P_d8["label"] = (P_d8["correct_choice_alphabet"] == P_d8["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P_d6 = pd.read_csv("./exps/outputs-ZS-train-n_drops=6-n_sample=1500.csv")
P_d6["n_drops"] = 6
P_d6["label"] = (P_d6["correct_choice_alphabet"] == P_d6["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P_d4 = pd.read_csv("./exps/outputs-ZS-train-n_drops=4-n_sample=1500.csv")
P_d4["n_drops"] = 4
P_d4["label"] = (P_d4["correct_choice_alphabet"] == P_d4["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P_d2 = pd.read_csv("./exps/outputs-ZS-train-n_drops=2-n_sample=2000.csv")
P_d2["n_drops"] = 2
P_d2["label"] = (P_d2["correct_choice_alphabet"] == P_d2["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P_d0 = pd.read_csv("./exps/outputs-ZS-train-n_drops=0-n_sample=1500.csv")
P_d0["n_drops"] = 0
P_d0["label"] = (P_d0["correct_choice_alphabet"] == P_d0["parsed_response"]).astype(int).apply(lambda x: "correct" if x == 1 else "incorrect")

P = pd.concat([P_d10, P_d8, P_d6, P_d4, P_d2, P_d0])

print(P.shape)
P.head(4)


(9500, 9)


,data_idx,question,prompt,response,correct_choice,correct_choice_alphabet,parsed_response,n_drops,label
0,3170,Why are these people on the backs of elephants?,Why are these people on the backs of elephants...,communismor otheriallys,transportation,D,A,10,incorrect
1,13220,Who holds an all-time record in a statistical ...,Who holds an all-time record in a statistical ...,veteraity_clusively,rickey henderson,B,A,10,incorrect
2,3478,What is one of the largest breeds of this animal?,What is one of the largest breeds of this anim...,veterin_ taxpayc,maine coon,A,B,10,incorrect
3,12996,What shape are the roofs?,What shape are the roofs?\nA. triangle\nB. squ...,communismist_iencedness,triangle,A,B,10,incorrect


In [15]:
V_text = SentenceTransformer("google/embeddinggemma-300m")

In [33]:
embs = []
for batch in tqdm(batchify_list(P["question"], 950)):
    embs.extend(V_text.encode_query(batch).tolist())

P["emb"] = embs


10it [00:18,  1.85s/it]


In [ ]:
accuracy_score()

In [51]:
 P["n_drops"]

0       10
1       10
2       10
3       10
4       10
        ..
1495     0
1496     0
1497     0
1498     0
1499     0
Name: n_drops, Length: 9500, dtype: int64

In [93]:
X_cols = [f"x{i+1}" for i in range(768)]
Y_cols = ["y"]
P_prep = pd.DataFrame(np.array(P["emb"].to_list()), columns=X_cols)
P_prep["w"] = P["n_drops"].to_list()
P_prep["y"] = P["label"].to_list()


P_moderators = P_prep[X_cols].mul(P_prep["w"], axis=0).rename(columns=lambda x: f"{x}.w")

P_prep = pd.concat([
    P_prep,
    P_moderators,
], axis=1)
W_cols = P_moderators.columns

P_prep.head(4)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x759.w,x760.w,x761.w,x762.w,x763.w,x764.w,x765.w,x766.w,x767.w,x768.w
0,-0.135377,0.019450,0.043551,0.006620,0.016625,-0.007755,-0.049687,0.004504,-0.031581,-0.025605,...,0.504061,-0.156696,-0.159190,-0.408389,0.398492,0.683270,-0.028180,0.220185,0.055287,0.457838
1,-0.109021,-0.020478,0.011604,-0.046108,-0.079904,-0.074321,0.044662,0.083459,-0.000972,-0.018367,...,-0.539427,-0.004304,0.125046,-0.214758,-0.265586,0.023967,0.696008,0.369896,-0.011161,-0.320005
2,-0.075372,-0.020491,-0.010681,-0.049102,-0.032288,-0.011896,-0.032107,0.076646,-0.015573,-0.020766,...,0.103719,-0.727391,-0.127336,0.366680,-0.267063,-0.189414,0.142421,0.281991,-0.168242,-0.434733
3,-0.140615,0.050464,-0.015612,0.030352,-0.021049,-0.010112,-0.039070,-0.009767,-0.051579,0.001010,...,-0.062624,0.041598,0.882523,-0.143277,-0.464682,0.301830,0.254161,0.113502,-0.036413,0.150959


In [96]:
P_prep_tr = P_prep.sample(frac=0.60).copy()
P_prep_te = P_prep.drop(P_prep_tr.index).copy()

In [97]:
P_prep_tr.shape

(5700, 1538)

In [98]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [100]:
regressor = LogisticRegression().fit(P_prep_tr[X_cols + ["w"]], P_prep_tr["y"])

print(classification_report(regressor.predict(P_prep_te[X_cols + ["w"]]), P_prep_te["y"]))

              precision    recall  f1-score   support

     correct       0.72      0.73      0.72      1665
   incorrect       0.78      0.78      0.78      2135

    accuracy                           0.76      3800
   macro avg       0.75      0.75      0.75      3800
weighted avg       0.76      0.76      0.76      3800



In [101]:
import joblib 

In [102]:
joblib.dump(regressor, "n_drop-predictor-v0.1.joblib")

['n_drop-predictor-v0.1.joblib']

In [60]:
P_prep_tr[mediator_names].to_series()

AttributeError: 'DataFrame' object has no attribute 'to_series'

In [ ]:


# Assume you have data: X (input), M_true (mediator labels), Y (outcome)

# Step 1: Train a model to predict the Mediator based on X
model_x_to_m = LinearRegression()
model_x_to_m.fit(P_prep_tr[independent_names], P_prep_tr[mediator_names])

# Step 2: Train a model to predict Y based on M (and optionally X)
# We use M_true for training to ensure the model learns the relationship correctly
model_m_to_y = LogisticRegression()
model_m_to_y.fit(P_prep_tr[mediator_names], P_prep_tr[dependent_names]) # Or use pandas to concat X and M_true if you want partial mediation

# --- PREDICTION TIME ---

# # 1. New data comes in
# new_X = [[...]]



/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [62]:
# 2. Predict the latent Mediator
predicted_m = model_x_to_m.predict(P_prep_te[independent_names])

# 3. Use the predicted M to predict Y
final_y = model_m_to_y.predict(predicted_m)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [66]:
predicted_m

array([[1.72563018],
       [5.90255692],
       [4.38953657],
       ...,
       [5.29921837],
       [4.82021181],
       [6.24906386]])

In [65]:
accuracy_score(final_y, P_prep_te[dependent_names])

0.5238390092879257

In [22]:
P["emb"] = [V_text.encode_query(q).tolist() for q in tqdm(P["question"])]

 12%|█▏        | 1119/9500 [00:58<07:19, 19.07it/s]


KeyboardInterrupt: 

In [20]:
V_text.encode_query("Hello, world!").tolist()

[-0.24900467693805695,
 0.02476084977388382,
 0.047170240432024,
 -0.04404875263571739,
 0.00038834940642118454,
 -0.013168609701097012,
 0.011229044757783413,
 -0.02068139798939228,
 -0.02557511068880558,
 -0.02331097610294819,
 -0.03098215162754059,
 -0.061816878616809845,
 0.014519142918288708,
 -0.022497225552797318,
 0.012144941836595535,
 0.036013610661029816,
 -0.042892392724752426,
 -0.035164181143045425,
 -0.01592344418168068,
 -0.05388514697551727,
 -0.0183764286339283,
 0.00470867520198226,
 -0.01015322096645832,
 -0.05743338540196419,
 0.03524608537554741,
 0.06811382621526718,
 0.015067395754158497,
 -0.01579853519797325,
 0.016612807288765907,
 0.04935219883918762,
 0.06450343877077103,
 0.006831655278801918,
 0.05521706864237785,
 0.005798820406198502,
 0.016934620216488838,
 0.016586922109127045,
 -0.011448448523879051,
 -0.03165535256266594,
 -0.02566245198249817,
 -0.02783123403787613,
 -0.05258937552571297,
 -0.01982567086815834,
 0.01145943347364664,
 -0.00344786047

In [ ]:
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [6]:
V_text.encode_query("Hello, world!")

array([-2.49004677e-01,  2.47608498e-02,  4.71702404e-02, -4.40487526e-02,
        3.88349406e-04, -1.31686097e-02,  1.12290448e-02, -2.06813980e-02,
       -2.55751107e-02, -2.33109761e-02, -3.09821516e-02, -6.18168786e-02,
        1.45191429e-02, -2.24972256e-02,  1.21449418e-02,  3.60136107e-02,
       -4.28923927e-02, -3.51641811e-02, -1.59234442e-02, -5.38851470e-02,
       -1.83764286e-02,  4.70867520e-03, -1.01532210e-02, -5.74333854e-02,
        3.52460854e-02,  6.81138262e-02,  1.50673958e-02, -1.57985352e-02,
        1.66128073e-02,  4.93521988e-02,  6.45034388e-02,  6.83165528e-03,
        5.52170686e-02,  5.79882041e-03,  1.69346202e-02,  1.65869221e-02,
       -1.14484485e-02, -3.16553526e-02, -2.56624520e-02, -2.78312340e-02,
       -5.25893755e-02, -1.98256709e-02,  1.14594335e-02, -3.44786048e-03,
       -2.17127986e-03, -5.75439408e-02, -9.35136750e-02, -1.13423420e-02,
       -3.50502469e-02, -3.48997861e-02,  2.23639589e-02,  9.60943103e-03,
       -1.85276903e-02,  

In [ ]:
V_text.device|

modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]